In [2]:
import numpy as np
import random
import json
import pandas as pd
import csv
from nltk.tokenize import sent_tokenize, word_tokenize
from evaluate import load

/home/shreya/.venvs/gen-3.9.7/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DATA_PATH = "../../data/10/dissim_dict.json"

In [34]:
data = json.load(open(DATA_PATH, "r"))
data_sz = len(data)
data_x = list(data.keys())
data_cand = list(data.values())
with open('../../data/10/labels.txt', 'r') as f:
    data_y = f.read().splitlines()

In [ ]:
class FeatureExtractor:
    @staticmethod
    def jaccard_similarity(cand, label) :
        cand_set, label_set = set(cand), set(label)
        intersection = cand_set & label_set
        union = cand_set | label_set
        return len(intersection) / len(union)

    @staticmethod
    def get_feature_vector(cand, src):

        features = []
        word_tok_src, word_tok_cand = word_tokenize(src.lower()), word_tokenize(cand.lower())
        wc_src, wc_cand = len(word_tok_src), len(word_tok_cand)
        sc_cand = len(sent_tokenize(cand))
        features.append(sc_cand * 1.0)
        features.append(wc_src * 1.0)
        features.append(FeatureExtractor.jaccard_similarity(word_tok_cand, word_tok_src))
        features.append(wc_cand * 1.0 / wc_src)
        features.append(wc_cand * 1.0 / sc_cand)

        return features

In [4]:
bertscore = load("bertscore")
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]
results = bertscore.compute(predictions=predictions, references=references, lang="en")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]
results = bertscore.compute(predictions=predictions, references=references, lang="en")

In [ ]:
class Ranker:
    def __init__(self, binner, model) -> None:
        self.binner = binner
        self.bertscore = load("bertscore")
        self.model = model


    @staticmethod
    def gold_score(cand, src, target, bertscore):
        bs = bertscore.compute(predictions=[cand], references=[src], lang="en")
        len_src = len(word_tokenize(src))
        phi_v = len(word_tokenize(cand)) * 1.0 / len_src
        phi_y = len(word_tokenize(target)) * 1.0 / len_src
        gs = np.exp(abs(phi_v - phi_y)) * bs
        return gs


    def gold_score(self, cand, src, target):
        bs = self.bertscore.compute(predictions=[cand], references=[target], lang="en")
        len_src = len(word_tokenize(src))
        phi_v = len(word_tokenize(cand)) * 1.0 / len_src
        phi_y = len(word_tokenize(target)) * 1.0 / len_src
        gs = np.exp(abs(phi_v - phi_y)) * bs
        return gs


    